In [ ]:
# ExtractROI - skip dummy scans
'''
To be checked whether it has to be used or not. It just skips some scans from the window.
'''
extract = Node(ExtractROI(t_min=4, t_size=-1),
               output_type='NIFTI_GZ',
               name="extract")


In [ ]:
if (MotionCorrection==1):
    # MCFLIRT - motion correction
    mcflirt = Node(MCFLIRT(mean_vol=True,
                           save_plots=True,
                           output_type='NIFTI_GZ'),
                   name="mcflirt")

In [ ]:
# Smooth - image smoothing
smooth = Node(Smooth(),
              fwhm = FWHM,
              name="smooth")


In [ ]:
# if highpass:
#     highpass = Node(interface=fsl.ImageMaths(suffix='_tempfilt'),
#                           name='highpass')
#     #featpreproc.connect(inputnode, ('highpass', highpass_operand), highpass, 'op_string')
#     featpreproc.connect(meanscale, 'out_file', highpass, 'in_file')

#     """
#     Add back the mean removed by the highpass filter operation as of FSL 5.0.7
#     """
#     meanfunc4 = Node(interface=fsl.ImageMaths(op_string='-Tmean',
#                                                     suffix='_mean'),
#                            name='meanfunc4')

#     featpreproc.connect(meanscale, 'out_file', meanfunc4, 'in_file')
#     addmean = Node(interface=fsl.BinaryMaths(operation='add'),
#                          iterfield=['in_file', 'operand_file'],
#                          name='addmean')
#     featpreproc.connect(highpass, 'out_file', addmean, 'in_file')
#     featpreproc.connect(meanfunc4, 'out_file', addmean, 'operand_file')
#     featpreproc.connect(addmean, 'out_file', outputnode, 'highpassed_files')

In [ ]:
art = Node(ArtifactDetect(norm_threshold=2,
                          zintensity_threshold=3,
                          mask_type='spm_global',
                          parameter_source='FSL',
                          use_differences=[True, False],
                          plot_type='svg'),
           name="art")

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['func', 'anat']),
                  name="infosource")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

## Use the following DataSink output substitutions
substitutions = [('_subject_id_', ''),
                 ('_task_name_', '/task-'),
                 ('_fwhm_', 'fwhm-'),
                 ('_roi', ''),
                 ('_mcf', ''),
                 ('_st', ''),
                 ('_flirt', ''),
                 ('.nii_mean_reg', '_mean'),
                 ('.nii.par', '.par'),
                 ]

datasink.inputs.substitutions = substitutions

In [ ]:


# Create a preprocessing workflow
preproc = Workflow(name='preproc')
preproc.base_dir = opj(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow
preproc.connect([(infosource, extract, [('func', 'in_file')]),
                 (extract, mcflirt, [('roi_file', 'in_file')]),
                 (mcflirt, slicetimer, [('out_file', 'in_file')]),

                 (infosource, coregwf, [('anat', 'bet_anat.in_file'),
                                         ('anat', 'coreg_bbr.reference')]),
                 (mcflirt, coregwf, [('mean_img', 'coreg_pre.in_file'),
                                     ('mean_img', 'coreg_bbr.in_file'),
                                     ('mean_img', 'applywarp_mean.in_file')]),
                 (slicetimer, coregwf, [('slice_time_corrected_file', 'applywarp.in_file')]),
                 
                 (coregwf, smooth, [('applywarp.out_file', 'in_files')]),
#                 (inputnode, highpass,[(('highpass', highpass_operand), 'op_string')]),
                 (mcflirt, datasink, [('par_file', 'preproc.@par')]),
                 (smooth, datasink, [('smoothed_files', 'preproc.@smooth')]),
                 
                 (coregwf, datasink, [('applywarp_mean.out_file', 'preproc.@mean')]),

                 (coregwf, art, [('applywarp.out_file', 'realigned_files')]),
                 (mcflirt, art, [('par_file', 'realignment_parameters')]),

                 (coregwf, datasink, [('coreg_bbr.out_matrix_file', 'preproc.@mat_file'),
                                      ('bet_anat.out_file', 'preproc.@brain')]),
                 (art, datasink, [('outlier_files', 'preproc.@outlier_files'),
                                  ('plot_files', 'preproc.@plot_files')]),
                 ])



In [ ]:


# Create preproc output graph
preproc.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename=opj(preproc.base_dir, 'preproc', 'graph.dot.png'))


In [ ]:
# Visualize the detailed graph
preproc.write_graph(graph2use='flat', format='png', simple_form=True)
Image(filename=opj(preproc.base_dir, 'preproc', 'graph_detailed.dot.png'))


In [ ]:
ab = fsl.ImageStats(op_string='-k %s -p 50', in_file = "/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3577037/session_1/rest_1/rest_linearMNI3mm.nii.gz",
                   mask_file = "/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3577037/session_1/rest_1/rest_linearMNI3mm.nii.gz")
ab.cmdline

<bound method Graph.size of <networkx.classes.digraph.DiGraph object at 0x7f7e8e16feb8>>

In [57]:
import os
import json
import nipype.interfaces.utility as util
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node, MapNode

from nipype.interfaces import fsl
import nipype.workflows.fmri.fsl.preprocess as Preprocessor

In [58]:
Reg_WorkFlow = Preprocessor.create_reg_workflow()

In [61]:
meanfunc = fsl.ImageMaths(op_string='-Tmean',
                                                    suffix='_mean', 
            in_file = '/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3577037/session_1/rest_1/rest_linearMNI3mm.nii.gz')
    

#Reg_WorkFlow.connect(meanfunc, 'out_file', Reg_WorkFlow.inputs.inputspec, 'mean_image')
Reg_WorkFlow.inputs.inputspec.mean_image = meanfunc.out_file

AttributeError: 'ImageMaths' object has no attribute 'out_file'

/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3999447/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3283638/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/2154093/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/1065809/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/4222184/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3334426/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/4098356/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/3003775/session_1/rest_1/rest_linearMNI3mm.nii.gz
/home/deepak/Desktop/funcConn/funcConnGUI/data/NYU_Cocaine/cocaine/1915879/session_1/rest_1/rest_linearMNI3mm.nii.gz

In [ ]:
import numpy as np
import sys
import json
import Preprocess_network as parallelPreproc
import timeit
import os
from shutil import copyfile

start = timeit.default_timer()
# JSONFile = sys.argv[1]
JSONFile = '/home/deepak/Desktop/FConnectivityAnalysis/FConnectivityAnalysisDesign.json'
with open(JSONFile) as JSON:
    try :
        
        data = json.load(JSON)
        AnalysisName = data['Analysis Name']
    #     print(AnalysisName)
        WorkingDir = data['WorkingDir']
        AnalysisParams = data['AnalysisParams']
        
    except :
        print('Unexpected error:', sys.exc_info()[0])
        raise

def get_TR(in_file):
    import nibabel
    f = nibabel.load(in_file)
    return f.get_header()['dim'][0]#['pixdim'][1:4].tolist(), f.get_header()['pixdim'][4]


def run_Preprocessing(AnalysisParams,FunctionalFiles,StructuralFiles,Group = 0):

    ReferenceFile = AnalysisParams['ReferSummary']['ReferImgPath']
    B0unwarping = AnalysisParams['B0 Unwarping']
    BETextract = AnalysisParams['BET Brain Extract']
    FWHM = AnalysisParams['FWHM']
    HighPass = AnalysisParams['High Pass']
    MotionCorrection = AnalysisParams['Motion Correction']
    Registration = AnalysisParams['Registration']
    SliceTimeCorrect = AnalysisParams['Slice Time Correct']
    Intensity_Norm = AnalysisParams['Intensity Normalization']
    MelodicICA = AnalysisParams['Melodic ICA']
    PerfusionSubtract = AnalysisParams['Perfusion Subtraction']
    OUTPUT_DIR = AnalysisParams['OutputInfo']['OutDirectory']
    TEMP_DIR_FOR_STORAGE = OUTPUT_DIR + '/tmp'
    FeatProcessName = 'featpreproc_group%s'%Group
    RegistrationName = 'registration_group%s'%Group
    RESULTS_FEAT_DATASINK = OUTPUT_DIR + '/tmp/%s/datasink/'%FeatProcessName
    TR = get_TR(FunctionalFiles[0])
    print('Using Repetition time: %s'%TR)
    if HighPass:
        preproc = parallelPreproc.create_parallelfeat_preproc(name = FeatProcessName,
                                    highpass= HighPass, 
                                    Intensity_Norm = Intensity_Norm,
                                    BETextract = BETextract,
                                    MotionCorrection = MotionCorrection, 
                                    SliceTimeCorrect = SliceTimeCorrect,
                                    time_repeat = TR)
        preproc.inputs.inputspec.highpass = 1/(0.09*TR)
        preproc.inputs.inputspec.func = FunctionalFiles
        preproc.inputs.inputspec.fwhm = FWHM
        preproc.base_dir = TEMP_DIR_FOR_STORAGE
        preproc.write_graph(graph2use='colored', format='png', simple_form=True)
        preproc.run('MultiProc', plugin_args={'n_procs': 4})
    else:
        preproc = parallelPreproc.create_parallelfeat_preproc(name = FeatProcessName,
                                    highpass= HighPass, 
                                    Intensity_Norm = Intensity_Norm,
                                    BETextract = BETextract,
                                    MotionCorrection = MotionCorrection, 
                                    SliceTimeCorrect = SliceTimeCorrect,
                                    time_repeat = TR)
        preproc.inputs.inputspec.func = FunctionalFiles
        preproc.inputs.inputspec.fwhm = FWHM
        preproc.base_dir = TEMP_DIR_FOR_STORAGE
        preproc.write_graph(graph2use='colored', format='png', simple_form=True)
        preproc.run('MultiProc', plugin_args={'n_procs': 4})

    datasink_results=[]
    datasink_results += [each for each in os.listdir(RESULTS_FEAT_DATASINK) if each.endswith('.json')]
    with open(RESULTS_FEAT_DATASINK + datasink_results[0]) as JSON:
        datafile = json.load(JSON)
        datafile =datafile[0][1][0][1]
    datasinkouts = []
    datasinkouts += [datafile[i][0] for i in range(len(datafile))]

    if Registration:
        datasinkouts_afterreg = []
        for i in range(len(datasinkouts)):
            RESULTS_REG_DATASINK = OUTPUT_DIR + '/tmp/%s_%s/datasink/'%(RegistrationName,i)
            Reg_WorkFlow = parallelPreproc.reg_workflow(name = '%s_%s'%(RegistrationName ,i))

            Reg_WorkFlow.inputs.inputspec.source_files = datasinkouts[i]
            Reg_WorkFlow.inputs.inputspec.anatomical_image = StructuralFiles[i]
            Reg_WorkFlow.inputs.inputspec.target_image = ReferenceFile
            Reg_WorkFlow.base_dir = TEMP_DIR_FOR_STORAGE
            regoutputs = Reg_WorkFlow.run()
            datasink_results=[]
            datasink_results += [each for each in os.listdir(RESULTS_REG_DATASINK) if each.endswith('.json')]
            with open(RESULTS_REG_DATASINK + datasink_results[0]) as JSON:
                datafile = json.load(JSON)
                datafile = datafile[0][1][0][1]
            
            datasinkouts_afterreg += [datafile[i][0] for i in range(len(datafile))]
            
        return datasinkouts_afterreg
    else:
        return datasinkouts

ReferenceFile = AnalysisParams['ReferSummary']['ReferImgPath']
ProcessingWay = AnalysisParams['ProcessingType']['ProcessingWay']
Ngroups = AnalysisParams['No of Groups']
if (ProcessingWay==0):
    FunctionaltxtFiles = AnalysisParams['FilesInfo']['FunctionalFilePaths']
    StructuraltxtFiles = AnalysisParams['FilesInfo']['StructuralFilePaths']

    for i in range(0,Ngroups):
        with open(FunctionaltxtFiles[i]) as file:
            FunctionalFiles_in_this_group = [line.strip('\n') for line in file]
        with open(StructuraltxtFiles[i]) as file:
            StructuralFiles_in_this_group = [line.strip('\n') for line in file]
        Preprocessed_Files = run_Preprocessing(AnalysisParams, FunctionalFiles_in_this_group, StructuralFiles_in_this_group,Group= i)
        print(Preprocessed_Files)

stop = timeit.default_timer()
print("Total time taken for running the program: ", stop - start)


In [165]:
from shutil import copyfile